In [ ]:
%load_ext autoreload
%autoreload 2

import os

os.environ["LLMSCOPE_CACHE_DIR"] = "/vol/bitbucket/ad5518/llmscope_cache"

# Pipeline Test

In [2]:
from llmscope.datasets.managers import AciBenchDatasetManager

aci_dataset_manager = AciBenchDatasetManager()

In [ ]:
class AciTaskPreprocessor:
    def __call__(self, dataset):
        # Limit to first 10 samples for testing
        return dataset["train"].select(range(10))


aci_task_preprocessor = AciTaskPreprocessor()

In [4]:
from llmscope.llms.managers import HuggingFaceLlmManager

llama_llm_manager = HuggingFaceLlmManager(
    model_name="meta-llama/Llama-3.1-8B-Instruct", quantization="none"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
class AciPromptFormatter:
    def __call__(self, dialogue, **kwargs):
        system_prompt = "You are an expert clinical assistant specialising in the creation of medically accurate summaries from a dialogue between the doctor and patient."
        user_prompt = f"""Your task is to generate a clinical note based on a conversation between a doctor and a patient. Use the following format for the clinical note:

1. **CHIEF COMPLAINT**: [Brief description of the main reason for the visit]
2. **HISTORY OF PRESENT ILLNESS**: [Summary of the patient’s current health status and any changes since the last visit]
3. **REVIEW OF SYSTEMS**: [List of symptoms reported by the patient]
4. **PHYSICAL EXAMINATION**: [Findings from the physical examination]
5. **RESULTS**: [Relevant test results]
6. **ASSESSMENT AND PLAN**: [Doctor’s assessment and plan for treatment or further testing]

**Conversation:**
{dialogue}

**Note:**
"""
        return [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]


aci_prompt_formatter = AciPromptFormatter()

In [6]:
from llmscope.pipelines import SimplePipeline

aci_pipeline = SimplePipeline(
    dataset_manager=aci_dataset_manager,
    task_preprocessor=aci_task_preprocessor,
    prompt_formatter=aci_prompt_formatter,
    llm_manager=llama_llm_manager,
)

In [7]:
outputs = aci_pipeline.run()

Generating outputs:   0%|          | 0/10 [00:00<?, ? examples/s]

## Original Dialogue

In [9]:
print(outputs[0]["dialogue"])

[doctor] hi , martha . how are you ?
[patient] i'm doing okay . how are you ?
[doctor] i'm doing okay . so , i know the nurse told you about dax . i'd like to tell dax a little bit about you , okay ?
[patient] okay .
[doctor] martha is a 50-year-old female with a past medical history significant for congestive heart failure , depression and hypertension who presents for her annual exam . so , martha , it's been a year since i've seen you . how are you doing ?
[patient] i'm doing well . i've been traveling a lot recently since things have , have gotten a bit lighter . and i got my , my vaccine , so i feel safer about traveling . i've been doing a lot of hiking . uh , went to washington last weekend to hike in northern cascades, like around the mount baker area .
[doctor] nice . that's great . i'm glad to hear that you're staying active , you know . i , i just love this weather . i'm so happy the summer is over . i'm definitely more of a fall person .
[patient] yes , fall foliage is the 

## Reference Note

In [12]:
print(outputs[0]["note"])

CHIEF COMPLAINT

Annual exam.

HISTORY OF PRESENT ILLNESS

Martha Collins is a 50-year-old female with a past medical history significant for congestive heart failure, depression, and hypertension who presents for her annual exam. It has been a year since I last saw the patient.

The patient has been traveling a lot recently since things have gotten a bit better. She reports that she got her COVID-19 vaccine so she feels safer about traveling. She has been doing a lot of hiking.

She reports that she is staying active. She has continued watching her diet and she is doing well with that. The patient states that she is avoiding salty foods that she likes to eat. She has continued utilizing her medications. The patient denies any chest pain, shortness of breath, or swelling in her legs.

Regarding her depression, she reports that she has been going to therapy every week for the past year. This has been really helpful for her. She denies suicidal or homicidal ideation.

The patient reports

## Generated Note

In [10]:
print(outputs[0]["output"])

**CHIEF COMPLAINT**: 50-year-old female presents for annual exam with no specific complaints.

**HISTORY OF PRESENT ILLNESS**: The patient, Martha, is a 50-year-old female with a past medical history significant for congestive heart failure, depression, and hypertension. She has been doing well, having recently traveled extensively and stayed active through hiking. She reports doing a good job managing her congestive heart failure by adhering to a low-sodium diet and taking her medications as prescribed. However, she has been experiencing some stress at work, which has caused her blood pressure to increase. She has been managing her depression through weekly therapy sessions and reports feeling well.

**REVIEW OF SYSTEMS**: The patient endorses nasal congestion due to allergies, but denies any other symptoms such as chest pain, shortness of breath, swelling in her legs, nausea, vomiting, or abdominal pain.

**PHYSICAL EXAMINATION**: Physical examination reveals a 3/6 systolic ejection 